<a href="https://colab.research.google.com/github/Iditc/NLP/blob/main/Markov_Model_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘edgar_allan_poe.txt’ already there; not retrieving.

File ‘robert_frost.txt’ already there; not retrieving.



In [54]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [55]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]

In [56]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [57]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [58]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [59]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [60]:
len(Ytrain), len(Ytest)

(1615, 539)

In [61]:
train_text[:5]

['and tell me whether',
 'and laugh but smile no more',
 'ill tell you whod remember heman lapish',
 'wed kept all these years between ourselves',
 'oh fly let us fly for we must']

In [62]:
Ytrain[:5]

[1, 0, 1, 1, 0]

In [63]:
idx = 1
word2idx = {'<unk>' : 0}

In [64]:
# populate word2idx
for text in train_text:
  tokens = text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token] = idx
      idx += 1

In [65]:
word2idx

{'<unk>': 0,
 'and': 1,
 'tell': 2,
 'me': 3,
 'whether': 4,
 'laugh': 5,
 'but': 6,
 'smile': 7,
 'no': 8,
 'more': 9,
 'ill': 10,
 'you': 11,
 'whod': 12,
 'remember': 13,
 'heman': 14,
 'lapish': 15,
 'wed': 16,
 'kept': 17,
 'all': 18,
 'these': 19,
 'years': 20,
 'between': 21,
 'ourselves': 22,
 'oh': 23,
 'fly': 24,
 'let': 25,
 'us': 26,
 'for': 27,
 'we': 28,
 'must': 29,
 'it': 30,
 'be': 31,
 'right': 32,
 'to': 33,
 'leave': 34,
 'him': 35,
 'i': 36,
 'cannot': 37,
 'write': 38,
 'speak': 39,
 'or': 40,
 'think': 41,
 'spirits': 42,
 'moving': 43,
 'musically': 44,
 'was': 45,
 'night': 46,
 'in': 47,
 'the': 48,
 'lonesome': 49,
 'october': 50,
 'god': 51,
 'man': 52,
 'beasts': 53,
 'sake': 54,
 'thought': 55,
 'growing': 56,
 'things': 57,
 'would': 58,
 'do': 59,
 'good': 60,
 'o': 61,
 'care': 62,
 'not': 63,
 'that': 64,
 'my': 65,
 'earthly': 66,
 'lot': 67,
 'journeyed': 68,
 'down': 69,
 'here': 70,
 'full': 71,
 'of': 72,
 'lank': 73,
 'shivery': 74,
 'halfdrowned

In [66]:
len(word2idx)

2502

In [67]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [68]:
train_text_int[100:105]

[[391, 18, 392, 393, 394, 395, 8, 9],
 [36, 45, 312, 396, 33, 250, 397],
 [152, 35, 127, 143, 378, 398, 195, 399, 33, 41],
 [48, 400, 72, 401],
 [48, 402, 72, 403, 194, 237, 72, 404]]

In [69]:
# idit A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones((V))

A1 = np.ones((V, V))
pi1 = np.ones((V))

In [70]:
# compute counts for A and Pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [71]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()


In [72]:
# log A and pi sinc we dont need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [73]:
# comput priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.33436532507739936, 0.6656346749226006)

In [74]:
from numpy.core.function_base import logspace
# build a classifer
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # its the first token
        logprob +=logpi[idx]
      else:
        logprob +=logA[last_idx, idx]

      # updata last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [75]:
# each array must be in order since classes are assumed to index thes ilsts
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [76]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9950464396284829


In [77]:
Ptest = clf.predict(test_text_int)
print(f"Train acc: {np.mean(Ptest == Ytest)}")

Train acc: 0.8218923933209648


In [78]:
from sklearn.metrics import confusion_matrix, f1_score

In [79]:
cm =  confusion_matrix(Ytrain, Ptrain)
cm

array([[ 532,    8],
       [   0, 1075]])

In [80]:
cm_test =  confusion_matrix(Ytest, Ptest)
cm_test

array([[ 92,  86],
       [ 10, 351]])

In [81]:
f1_score(Ytrain, Ptrain)

0.9962928637627432

In [82]:
f1_score(Ytest, Ptest)

0.8796992481203008